## Many to one rnn (Sentiment Analysis)

### Datasets

In [114]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

# Here true is positive sentiments and False is negative sentiments

In [115]:
test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}

# Import libraries

In [116]:
import numpy as np
import random
from numpy.random import randn

# Create a dictionary for vocabulary

In [117]:
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))

In [118]:
vocab_size = len(vocab)
print(vocab_size)
vocab # so there is 18 unique word.

18


['not',
 'or',
 'i',
 'was',
 'bad',
 'now',
 'all',
 'right',
 'happy',
 'earlier',
 'good',
 'sad',
 'this',
 'am',
 'is',
 'at',
 'very',
 'and']

In [119]:
# Assign interger index to each word
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }


In [120]:
word_to_idx

{'not': 0,
 'or': 1,
 'i': 2,
 'was': 3,
 'bad': 4,
 'now': 5,
 'all': 6,
 'right': 7,
 'happy': 8,
 'earlier': 9,
 'good': 10,
 'sad': 11,
 'this': 12,
 'am': 13,
 'is': 14,
 'at': 15,
 'very': 16,
 'and': 17}

In [121]:
idx_to_word

{0: 'not',
 1: 'or',
 2: 'i',
 3: 'was',
 4: 'bad',
 5: 'now',
 6: 'all',
 7: 'right',
 8: 'happy',
 9: 'earlier',
 10: 'good',
 11: 'sad',
 12: 'this',
 13: 'am',
 14: 'is',
 15: 'at',
 16: 'very',
 17: 'and'}

In [122]:
# Now we can call any word in rnn using index value and each word has unique index value. RNN can not understand
# text so it is important

### finally recall each x_i input as vector
one hot vector incoding i am going to do here all element will be zero except for one 
Since we have 18 unique words so the size of one hot vector will be [18,1]

In [123]:
def createInputs(text):
    inputs = []
    for w in text.split(' '):
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        inputs.append(v)
    return inputs


### Forward propogation and back propogation through time
Now initialize the weight and biases

In [124]:
class RNN:
    # we are building Vanilla Recurrent neural networks, Vanilla rnn represent rnn has only one hidden layer.
    
    def __init__(self, input_size, output_size, hidden_size=64):
        # weights
        self.Whh = randn(hidden_size, hidden_size) / 1000
        self.Wxh = randn(hidden_size, input_size) / 1000
        self.Why = randn(output_size, hidden_size) / 1000
        
        # biases 
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))
        
        #for back propogation-----
        self.last_inputs = inputs
        self.last_hs = { 0: h }
        #-------------------------
        
        for i, x in enumerate(inputs):
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            self.last_hs[i + 1] = h
            
        y = self.Why @ h + self.by
        
        return y, h  
    
    def backprop(self, d_y, learn_rate=2e-2):
        n = len(self.last_inputs)
        #Calculate dL/dWhy and dL/dby
        d_Why = d_y @ self.last_hs[n].T
        d_by = d_y
        
        # Initialize dL/dWhh, dL/dWxh, and dL/dbh to zero.
        
        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh = np.zeros(self.bh.shape)
        
        # Calculate dL/dh for the last h
        d_h = self.Why.T @ d_y
        
        # Back propogation through time
        for t in reversed(range(n)):
            # An intermediate value 
            temp = ((1 - self.last_hs[t+1] ** 2) * d_h)
            d_bh += temp
            d_Whh += temp @ self.last_hs[t].T
            d_Wxh += temp @ self.last_inputs[t].T
            
            d_h = self.Whh @ temp
            
        #clip to prevent exploding gradient
        
        for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
            np.clip(d, -1, 1, out=d)
            
        # Update weights and biases using gradient descent
        self.Whh -= learn_rate * d_Whh
        self.Wxh -= learn_rate * d_Wxh
        self.Why -= learn_rate * d_Why
        self.bh -= learn_rate * d_bh
        self.by -= learn_rate * d_by
        
        
        

In [125]:
def softmax(xs):
    return np.exp(xs) / sum(np.exp(xs))
#initialize rnn



In [126]:
rnn = RNN(vocab_size, 2)


In [128]:
# loop for each training example

for x, y in train_data.items():
    inputs = createInputs(x)
    target = int(y)
    
    #Forward
    out, _ = rnn.forward(inputs)
    probs = softmax(out)
    
    # Build dL/dy
    d_L_d_y = probs
    d_L_d_y[target] -= 1
    
    rnn.backprop(d_L_d_y)

In [129]:
def processData(data, backprop=True):
    items = list(data.items())
    random.shuffle(items)
    loss = 0
    num_correct = 0
    
    for x, y in items:
        inputs = createInputs(x)
        target = int(y)
        
        # Forward
        out, _ = rnn.forward(inputs)
        probs = softmax(out)
        
        # Calculate loss / accuracy
        loss -= np.log(probs[target])
        num_correct += int(np.argmax(probs) == target)
        
        if backprop:
            # Build dL/dy
            d_L_d_y = probs
            d_L_d_y[target] -= 1
            # Backward
            rnn.backprop(d_L_d_y)
            
    return loss / len(data), num_correct / len(data)


In [130]:
# Training loop
for epoch in range(1000):
  train_loss, train_acc = processData(train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

    test_loss, test_acc = processData(test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.688 | Accuracy: 0.569
Test:	Loss 0.699 | Accuracy: 0.500
--- Epoch 200
Train:	Loss 0.660 | Accuracy: 0.672
Test:	Loss 0.715 | Accuracy: 0.450
--- Epoch 300
Train:	Loss 0.661 | Accuracy: 0.586
Test:	Loss 0.695 | Accuracy: 0.500
--- Epoch 400
Train:	Loss 0.155 | Accuracy: 0.948
Test:	Loss 0.102 | Accuracy: 1.000
--- Epoch 500
Train:	Loss 0.008 | Accuracy: 1.000
Test:	Loss 0.304 | Accuracy: 0.950
--- Epoch 600
Train:	Loss 0.004 | Accuracy: 1.000
Test:	Loss 0.280 | Accuracy: 0.950
--- Epoch 700
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.402 | Accuracy: 0.950
--- Epoch 800
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.436 | Accuracy: 0.950
--- Epoch 900
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.432 | Accuracy: 0.950
--- Epoch 1000
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.444 | Accuracy: 0.950
